In [31]:
from google.cloud import discoveryengine_v1beta as discoveryengine_v1
from google.protobuf.json_format import MessageToDict

project_id =328583281153
location_global = 'global'
collection = 'default_collection'
# data_store = 'film-metadata-b2_1709192252087'
data_store = 'film-metadata-2024-03-19_1710826473633'

In [130]:

import requests
import json
def vertex_search(query):
    client = discoveryengine_v1.SearchServiceClient()

    request = discoveryengine_v1.SearchRequest(
        serving_config=f"projects/{project_id}/locations/{location_global}/collections/{collection}/dataStores/{data_store}/servingConfigs/default_search:search",
        query=query,
        page_size=1
    )

    page_result = client.search(request=request)

    return page_result
    # result = list(map(lambda x: MessageToDict(x.document._pb)['structData'],page_result))
    # return result

In [153]:
import google.auth
import google.auth.transport.requests
# creds, project = google.auth.default()
# creds= google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

# auth_req = google.auth.transport.requests.Request()
# creds.refresh(auth_req)

from google.auth import compute_engine

cred = compute_engine.Credentials()


In [159]:
cred.token

In [137]:
data_store = "film-metadata-2024-03-19_1710826473633"
project_id = "328583281153"
location = "global"
token = "ya29.a0Ad52N3_2HpiiLqc-1UTWCejoyhxjZnVbgJ0eiu1pGqLrXNnBprcPdlEeaU9EHIOj1YEJqWgliL8sFdOQQPuTAAK7zUzrTM21iBfVE38pslYnMXcPNEPN2tDcLcx6y98IJScAcBLs7tnOLPiCESrUaMdDyzUjyWROoiZlgpfINXcaCgYKASkSARISFQHGX2MiZuk121rhEB8YTWN1rmr-tw0178"

def vertex_search_http(query):
    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{data_store}/servingConfigs/default_search:search"
    data = {"query": query, "pageSize": 10}
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.post(url, json=data, headers=headers)
    response_json = json.loads(response.text)

    result = list(map(lambda x: x["document"]["structData"], response_json["results"]))
    ids = list(map(lambda x: int(x['id']), result))

    return ids

In [139]:
vertex_search_http('ratu adil')

[9372, 6147, 6100, 6701, 3672, 629, 2593, 731, 9060, 3965]

In [104]:
vertex_search('ratu adil')

ResourceExhausted: 429 Quota exceeded for quota metric 'Search requests' and limit 'Search requests per minute' of service 'discoveryengine.googleapis.com' for consumer 'project_number:328583281153'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "discoveryengine.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "discoveryengine.googleapis.com/search_requests"
}
metadata {
  key: "quota_location"
  value: "global"
}
metadata {
  key: "quota_limit"
  value: "SearchRequestsPerMinutePerProject"
}
metadata {
  key: "quota_limit_value"
  value: "300"
}
metadata {
  key: "consumer"
  value: "projects/328583281153"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]

In [4]:
import vertexai
from vertexai.language_models import TextEmbeddingModel
from annoy import AnnoyIndex
import pandas as pd

class Embedding:
    def __init__(self, model) -> None:
        self.model = model

    def embedding_text(self, text):
        embeddings = self.model.get_embeddings([text])
        for embedding in embeddings:
            vector = embedding.values
            # print(f"Length of Embedding Vector: {len(vector)}")
        return vector

class FilmVertexSearch:
    def __init__(self, embedding:Embedding) -> None:
        self.embedding = embedding
        self.final_df = pd.read_json('./data/film_metadata.json', lines=True)
        self.annoy_index = AnnoyIndex(768, 'dot')
        for index, row in self.final_df.iterrows():
            self.annoy_index.add_item(row['id'], row['embedding'])
        self.annoy_index.build(10)

    def search(self, query, limit=10):
        project_id =328583281153
        location_global = 'global'
        collection = 'default_collection'
        data_store = 'film-metadata-b2_1709192252087'

        client = discoveryengine_v1.SearchServiceClient()

        # Initialize request argument(s)
        request = discoveryengine_v1.SearchRequest(
            serving_config=f"projects/{project_id}/locations/{location_global}/collections/{collection}/dataStores/{data_store}/servingConfigs/default_search:search",
            query=query,
            page_size=10
        )

        page_result = client.search(request=request)

        result = list(map(lambda x: MessageToDict(x.document._pb)['structData'],page_result))
        result_ids = list(map(lambda x: int(x['id']),result))
        return result_ids
        # query_vector = self.embedding.embedding_text(query)
        # return self.annoy_index.get_nns_by_vector(query_vector, limit, search_k=-1)

    def lookup_film(self, film_ids):
        return "\n\n".join([search_text for search_text in self.final_df[self.final_df['id'].isin(film_ids)]['search_text'].values.tolist()])

    def lookup_film_complete_text(self, film_ids):
        film_search_text = []
        for id, search_text in self.final_df[self.final_df['id'].isin(film_ids)][['id', 'search_text']].values.tolist():
            film_search_text.append(f"ID: {str(id)}\n{search_text}")
        return "\n\n".join(film_search_text)

    def get_grounding(self, response, history_film_ids):
        vector_search_result = ""

        if len(response) != 0:
            vector_film_ids = self.search(response)
            vector_search_result = self.lookup_film_complete_text(vector_film_ids)
        else:
            if len(history_film_ids) > 0:
                film_ids = []
                vector_search_result
                for film_id in history_film_ids:
                    film_search_query = self.lookup_film([film_id])
                    if film_search_query != "":
                        film_ids += self.search(film_search_query)
                vector_search_result = self.lookup_film_complete_text(set(film_ids))
        return vector_search_result

embedding_model = TextEmbeddingModel.from_pretrained("textembedding-gecko-multilingual")
embedding = Embedding(embedding_model)
film_vertex_search = FilmVertexSearch(embedding)

In [6]:
print(film_vertex_search.lookup_film(film_vertex_search.search("dian sastrowardoyo")))

ResourceExhausted: 429 Quota exceeded for quota metric 'Search requests' and limit 'Search requests per minute' of service 'discoveryengine.googleapis.com' for consumer 'project_number:328583281153'. [reason: "RATE_LIMIT_EXCEEDED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "discoveryengine.googleapis.com"
}
metadata {
  key: "quota_metric"
  value: "discoveryengine.googleapis.com/search_requests"
}
metadata {
  key: "quota_location"
  value: "global"
}
metadata {
  key: "quota_limit"
  value: "SearchRequestsPerMinutePerProject"
}
metadata {
  key: "quota_limit_value"
  value: "300"
}
metadata {
  key: "consumer"
  value: "projects/328583281153"
}
, links {
  description: "Request a higher quota limit."
  url: "https://cloud.google.com/docs/quota#requesting_higher_quota"
}
]